In [12]:
#!/usr/bin/env python3
import pandas as pd
import numpy as np

from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
import astropy.units as u

# allow unlimited rows per query
Gaia.ROW_LIMIT = -1

# read your input
df_targets = pd.read_csv('section11(Sheet1).csv')  # columns: RAdeg, DEdeg

output_rows = []

for _, row in df_targets.iterrows():
    ra0 = row['RAdeg']
    dec0 = row['DEdeg']
    target_coord = SkyCoord(ra=ra0*u.deg, dec=dec0*u.deg, frame='icrs')
    
    # 2″ cone-search
    job = Gaia.cone_search_async(target_coord, radius=2*u.arcsec, verbose=False)
    gaia_tbl = job.get_results()
    
    out = {'RAdeg': ra0, 'DEdeg': dec0}
    
    if len(gaia_tbl) == 0:
        # no match → fill your chosen Gaia columns with NaN
        for col in ['source_id','ra','dec','parallax','pmra','pmdec','phot_g_mean_mag', 'bp_rp']:
            out[col] = np.nan
    else:
        # convert MaskedColumn → plain numpy array of floats
        ra_vals  = np.array(gaia_tbl['ra'])
        dec_vals = np.array(gaia_tbl['dec'])
        
        gaia_coords = SkyCoord(ra=ra_vals*u.deg,
                               dec=dec_vals*u.deg,
                               frame='icrs')
        sep = gaia_coords.separation(target_coord)
        best = gaia_tbl[np.argmin(sep)]
        
        # copy *all* returned Gaia columns
        for c in best.colnames:
            out[c] = best[c]
    
    output_rows.append(out)

# build output DataFrame and write it out
df_out = pd.DataFrame(output_rows)
df_out.to_csv('gaia_bulk_crossmatch_results_11.csv', index=False)

print(f"Wrote {len(df_out)} rows to gaia_bulk_crossmatch_results_11.csv")


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
